In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [104]:
df = spark.read.csv('student_marks.csv', header=True)
df.show()

+--------+------+----------+-----+-------+---------+-------+-------+---------+-------+------+
|     _c0|Gender|       DOB|Maths|Physics|Chemistry|English|Biology|Economics|History|Civics|
+--------+------+----------+-----+-------+---------+-------+-------+---------+-------+------+
|    John|     M|05/04/1988|   55|     45|       56|     87|     21|       52|     89|    65|
|  Suresh|     M|  4/5/1987|   75|     55|     null|     64|     90|       61|     58|     2|
|  Ramesh|     M| 25/5/1989|   25|     54|       89|     76|     95|       87|     56|    74|
| Jessica|     F| 12/8/1990|   78|     55|       86|     63|     54|       89|     75|    45|
|Jennifer|     F|  2/9/1989|   58|     96|       78|     46|     96|       77|     83|    53|
+--------+------+----------+-----+-------+---------+-------+-------+---------+-------+------+



In [105]:
df.describe().show(vertical=True)

-RECORD 0-----------------------
 summary   | count              
 _c0       | 5                  
 Gender    | 5                  
 DOB       | 5                  
 Maths     | 5                  
 Physics   | 5                  
 Chemistry | 4                  
 English   | 5                  
 Biology   | 5                  
 Economics | 5                  
 History   | 5                  
 Civics    | 5                  
-RECORD 1-----------------------
 summary   | mean               
 _c0       | null               
 Gender    | null               
 DOB       | null               
 Maths     | 58.2               
 Physics   | 61.0               
 Chemistry | 77.25              
 English   | 67.2               
 Biology   | 71.2               
 Economics | 73.2               
 History   | 72.2               
 Civics    | 47.8               
-RECORD 2-----------------------
 summary   | stddev             
 _c0       | null               
 Gender    | null               
 DOB      

# filter()

In [19]:
students = df.select('_c0').filter(df.Maths>'50')
students.show(10)

+--------+
|     _c0|
+--------+
|    John|
|  Suresh|
| Jessica|
|Jennifer|
+--------+



# ReduceBykey

In [21]:

data = [('Project', 1),
('Gutenberg’s', 1),
('Alice’s', 1),
('Adventures', 1),
('in', 1),
('Wonderland', 1),
('Project', 1),
('Gutenberg’s', 1),
('Adventures', 1),
('in', 1),
('Wonderland', 1),
('Project', 1),
('Gutenberg’s', 1)]

rdd=spark.sparkContext.parallelize(data)
rdd.collect()

[('Project', 1),
 ('Gutenberg’s', 1),
 ('Alice’s', 1),
 ('Adventures', 1),
 ('in', 1),
 ('Wonderland', 1),
 ('Project', 1),
 ('Gutenberg’s', 1),
 ('Adventures', 1),
 ('in', 1),
 ('Wonderland', 1),
 ('Project', 1),
 ('Gutenberg’s', 1)]

In [22]:
rdd2=rdd.reduceByKey(lambda a,b: a+b)
for element in rdd2.collect():
    print(element)

('Gutenberg’s', 3)
('Alice’s', 1)
('in', 2)
('Adventures', 2)
('Wonderland', 2)
('Project', 3)


# groupByKey()

In [29]:
data = spark.sparkContext.parallelize([
    ("USA", 1), ("USA", 2), ("India", 1),
    ("UK", 1), ("India", 4), ("India", 9),
    ("USA", 8), ("USA", 3), ("India", 4),
    ("UK", 6), ("UK", 9), ("UK", 5)], 3) 
x = data.groupByKey()
x.collect()

[('USA', <pyspark.resultiterable.ResultIterable at 0x7f86d338f2e8>),
 ('UK', <pyspark.resultiterable.ResultIterable at 0x7f86d338f320>),
 ('India', <pyspark.resultiterable.ResultIterable at 0x7f86d338f3c8>)]

In [30]:
x.getNumPartitions()

3

In [31]:
y = data.groupByKey(2)
y.collect()

[('USA', <pyspark.resultiterable.ResultIterable at 0x7f86d338ff98>),
 ('India', <pyspark.resultiterable.ResultIterable at 0x7f86d3317080>),
 ('UK', <pyspark.resultiterable.ResultIterable at 0x7f86d33170b8>)]

In [32]:
for t in y.collect():
    print(t[0], [v for v in t[1]])

USA [1, 2, 8, 3]
India [1, 4, 9, 4]
UK [1, 6, 9, 5]


# Repartition

In [33]:
df.rdd.getNumPartitions()

1

In [39]:
df = df.repartition(5)

In [40]:
df.rdd.getNumPartitions()


5

In [76]:
df1 = spark.read.csv('Case.csv', header=True)
df1.show(1)

+--------+--------+----------+-----+--------------+---------+---------+----------+
| case_id|province|      city|group|infection_case|confirmed| latitude| longitude|
+--------+--------+----------+-----+--------------+---------+---------+----------+
| 1000001|   Seoul|Yongsan-gu| TRUE| Itaewon Clubs|      139|37.538621|126.992652|
+--------+--------+----------+-----+--------------+---------+---------+----------+
only showing top 1 row



## distint()

In [55]:
df1.select('province', 'city').distinct().show()

+----------------+---------------+
|        province|           city|
+----------------+---------------+
|Gyeongsangnam-do|       Jinju-si|
|           Seoul|        Guro-gu|
|           Seoul|     Gangnam-gu|
|         Daejeon|              -|
|    Jeollabuk-do|from other city|
|Gyeongsangnam-do|Changnyeong-gun|
|           Seoul|              -|
|         Jeju-do|from other city|
|Gyeongsangbuk-do|              -|
|Gyeongsangnam-do|   Geochang-gun|
|Gyeongsangbuk-do|        Gumi-si|
|         Incheon|from other city|
|           Busan|              -|
|           Daegu|         Seo-gu|
|           Busan|     Suyeong-gu|
|     Gyeonggi-do|   Uijeongbu-si|
|           Seoul|     Yongsan-gu|
|           Daegu|              -|
|           Seoul|   Seodaemun-gu|
|     Gyeonggi-do|    Seongnam-si|
+----------------+---------------+
only showing top 20 rows



# sample()

In [57]:
sampledata =  df1.sample(False, 0.2, 42)
sampledata.show(), sampledata.count()

+--------+-----------+---------------+-----+--------------------+---------+----------+-----------+
| case_id|   province|           city|group|      infection_case|confirmed|  latitude|  longitude|
+--------+-----------+---------------+-----+--------------------+---------+----------+-----------+
| 1000008|      Seoul|  Dongdaemun-gu| TRUE|       Dongan Church|       17| 37.592888| 127.056766|
| 1000017|      Seoul|      Jongno-gu| TRUE|Korea Campus Crus...|        7| 37.594782| 126.968022|
| 1000019|      Seoul|from other city| TRUE|Daejeon door-to-d...|        1|         -|          -|
| 1000027|      Seoul|      Seocho-gu| TRUE|       Seocho Family|        5|         -|          -|
| 1000037|      Seoul|              -|FALSE|contact with patient|      162|         -|          -|
| 1100001|      Busan|     Dongnae-gu| TRUE|       Onchun Church|       39|  35.21628|   129.0771|
| 1200001|      Daegu|         Nam-gu| TRUE|  Shincheonji Church|     4511|  35.84008|   128.5667|
| 1200003|

(None, 33)

In [58]:
sampledata.cache()

DataFrame[ case_id: string, province: string, city: string, group: string, infection_case: string, confirmed: string, latitude: string, longitude: string]

## Union

In [70]:
simpleData = [("James","Sales","NY",90000,34,10000), \
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000) \
  ]
columns= ["employee_name","department","state","salary","age","bonus"]
df11 = spark.createDataFrame(data = simpleData, schema = columns)
df11.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
+-------------+----------+-----+------+---+-----+



In [71]:
simpleData2 = [("James","Sales","NY",90000,34,10000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]
columns2= ["employee_name","department","state","salary","age","bonus"]

df22 = spark.createDataFrame(data = simpleData2, schema = columns2)
df22.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [72]:
union_df = df11.union(df22)
union_df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        James|     Sales|   NY| 90000| 34|10000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



## Intersection 


In [73]:
intersection_df = df11.intersect(df22)
intersection_df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        Maria|   Finance|   CA| 90000| 24|23000|
|        James|     Sales|   NY| 90000| 34|10000|
+-------------+----------+-----+------+---+-----+



# Map

In [82]:
df1 = df1.select('city')
rdd2=df1.rdd.map(lambda x: (x,1))
rdd2.collect()

[(Row(city='Yongsan-gu'), 1),
 (Row(city='Gwanak-gu'), 1),
 (Row(city='Guro-gu'), 1),
 (Row(city='Yangcheon-gu'), 1),
 (Row(city='Dobong-gu'), 1),
 (Row(city='Guro-gu'), 1),
 (Row(city='from other city'), 1),
 (Row(city='Dongdaemun-gu'), 1),
 (Row(city='from other city'), 1),
 (Row(city='Gwanak-gu'), 1),
 (Row(city='Eunpyeong-gu'), 1),
 (Row(city='Seongdong-gu'), 1),
 (Row(city='Jongno-gu'), 1),
 (Row(city='Gangnam-gu'), 1),
 (Row(city='Jung-gu'), 1),
 (Row(city='Seodaemun-gu'), 1),
 (Row(city='Jongno-gu'), 1),
 (Row(city='Gangnam-gu'), 1),
 (Row(city='from other city'), 1),
 (Row(city='Geumcheon-gu'), 1),
 (Row(city='from other city'), 1),
 (Row(city='from other city'), 1),
 (Row(city='Jung-gu'), 1),
 (Row(city='Yeongdeungpo-gu'), 1),
 (Row(city='Gangnam-gu'), 1),
 (Row(city='Yangcheon-gu'), 1),
 (Row(city='Seocho-gu'), 1),
 (Row(city='from other city'), 1),
 (Row(city='Gangnam-gu'), 1),
 (Row(city='Gangseo-gu'), 1),
 (Row(city='from other city'), 1),
 (Row(city='Jung-gu'), 1),
 (Row(

In [86]:
# df1 = df1.select('city')
# rdd3 = df1.rdd.flatmap(lambda x: x.split(x,1))
# rdd3.collect()

In [100]:
totalLength = rdd2.reduce(lambda a, b: a + b)
totalLength

(Row(city='Yongsan-gu'),
 1,
 Row(city='Gwanak-gu'),
 1,
 Row(city='Guro-gu'),
 1,
 Row(city='Yangcheon-gu'),
 1,
 Row(city='Dobong-gu'),
 1,
 Row(city='Guro-gu'),
 1,
 Row(city='from other city'),
 1,
 Row(city='Dongdaemun-gu'),
 1,
 Row(city='from other city'),
 1,
 Row(city='Gwanak-gu'),
 1,
 Row(city='Eunpyeong-gu'),
 1,
 Row(city='Seongdong-gu'),
 1,
 Row(city='Jongno-gu'),
 1,
 Row(city='Gangnam-gu'),
 1,
 Row(city='Jung-gu'),
 1,
 Row(city='Seodaemun-gu'),
 1,
 Row(city='Jongno-gu'),
 1,
 Row(city='Gangnam-gu'),
 1,
 Row(city='from other city'),
 1,
 Row(city='Geumcheon-gu'),
 1,
 Row(city='from other city'),
 1,
 Row(city='from other city'),
 1,
 Row(city='Jung-gu'),
 1,
 Row(city='Yeongdeungpo-gu'),
 1,
 Row(city='Gangnam-gu'),
 1,
 Row(city='Yangcheon-gu'),
 1,
 Row(city='Seocho-gu'),
 1,
 Row(city='from other city'),
 1,
 Row(city='Gangnam-gu'),
 1,
 Row(city='Gangseo-gu'),
 1,
 Row(city='from other city'),
 1,
 Row(city='Jung-gu'),
 1,
 Row(city='from other city'),
 1,
 Row

In [102]:
# Action resturns the first row 
df.first()

Row(firstname='James', lastname='Smith', gender='M', salary=3000)